In [2]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import joblib
import numpy as np

app = Flask(__name__)
CORS(app)

print("\n" + "="*70)
print("🍽️  RESTAURANT ORDER PREDICTOR API")
print("="*70)

try:
    model = joblib.load('D:\\Rachith Bharadwaj T N 24BDS062\\2nd Year\\3rd SEM\\DHV Project\\restaurant-order-prediction\\backend\\nb_model.pkl')
    encoders = joblib.load('D:\\Rachith Bharadwaj T N 24BDS062\\2nd Year\\3rd SEM\\DHV Project\\restaurant-order-prediction\\backend\\encoders.pkl')
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Error: {e}")
    model = None
    encoders = None

@app.route('/api/health', methods=['GET'])
def health():
    return jsonify({'status': 'healthy', 'model_loaded': model is not None}), 200

@app.route('/api/predict', methods=['POST', 'OPTIONS'])
def predict():
    if request.method == 'OPTIONS':
        return '', 204
    
    try:
        data = request.get_json()
        # Extract only the 4 FEATURES (NOT the target 'order')
        cuisine = data.get('cuisine')
        time_of_day = data.get('timeOfDay')
        weather = data.get('weather')
        hunger_level = data.get('hungerLevel')
        
        print(f"\nReceived: cuisine={cuisine}, time={time_of_day}, weather={weather}, hunger={hunger_level}")
        
        if not all([cuisine, time_of_day, weather, hunger_level]):
            return jsonify({'error': 'Missing fields'}), 400
        
        if model is None:
            return jsonify({'error': 'Model not loaded'}), 500
        
        # Correct encoding: each .transform returns a single value, append that, NOT the array.
        encoded = [
            encoders['cuisine'].transform([cuisine])[0],
            encoders['time'].transform([time_of_day])[0],
            encoders['weather'].transform([weather])[0],
            encoders['hunger'].transform([hunger_level])[0]
        ]
        
        print(f"Encoded: {encoded}")
        
        # [encoded] → shape (1, 4) as expected
        X = np.array([encoded])
        print(f"Shape: {X.shape}")
        
        prediction = model.predict(X)[0]  # single value
        proba = model.predict_proba(X)[0] # 2 probabilities: [no, yes]
        
        print(f"Prediction: {prediction}, Proba: {proba}")
        
        # Return the class label using label encoder, NOT just 'Yes'/'No'!
        pred_text = encoders['order'].inverse_transform([prediction])[0]
        confidence = float(max(proba) * 100)
        
        return jsonify({
            'prediction': pred_text,
            'confidence': confidence,
            'yes_probability': float(proba[1] * 100) if len(proba) > 1 else 0,
            'no_probability': float(proba[0] * 100) if len(proba) > 0 else 0,
            'input': {
                'cuisine': cuisine,
                'timeOfDay': time_of_day,
                'weather': weather,
                'hungerLevel': hunger_level
            },
            'feature_importance': {
                'Cuisine': 0.25,
                'TimeOfDay': 0.25,
                'Weather': 0.25,
                'HungerLevel': 0.25
            },
            'success': True
        }), 200

    except Exception as e:
        print(f"Error: {e}")
        return jsonify({'error': str(e)}), 500


if __name__ == '__main__':
    print("\n✅ Ready!")
    print("📍 http://localhost:5000")
    print("="*70 + "\n")
    app.run(host='127.0.0.1', port=5000, debug=False, use_reloader=False)



🍽️  RESTAURANT ORDER PREDICTOR API
✅ Model loaded successfully!

✅ Ready!
📍 http://localhost:5000

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Nov/2025 16:02:48] "OPTIONS /api/predict HTTP/1.1" 204 -
c:\Users\RACHITH BHARADWAJ T\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
c:\Users\RACHITH BHARADWAJ T\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
127.0.0.1 - - [15/Nov/2025 16:02:49] "POST /api/predict HTTP/1.1" 200 -



Received: cuisine=Indian, time=Lunch, weather=Cloudy, hunger=Medium
Encoded: [np.int64(1), np.int64(3), np.int64(0), np.int64(2)]
Shape: (1, 4)
Prediction: 0, Proba: [0.60694376 0.39305624]


127.0.0.1 - - [15/Nov/2025 16:05:36] "OPTIONS /api/predict HTTP/1.1" 204 -
c:\Users\RACHITH BHARADWAJ T\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
c:\Users\RACHITH BHARADWAJ T\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
127.0.0.1 - - [15/Nov/2025 16:05:36] "POST /api/predict HTTP/1.1" 200 -



Received: cuisine=Indian, time=Dinner, weather=Sunny, hunger=Low
Encoded: [np.int64(1), np.int64(1), np.int64(2), np.int64(1)]
Shape: (1, 4)
Prediction: 0, Proba: [0.84554114 0.15445886]


127.0.0.1 - - [15/Nov/2025 16:10:41] "OPTIONS /api/predict HTTP/1.1" 204 -
c:\Users\RACHITH BHARADWAJ T\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
c:\Users\RACHITH BHARADWAJ T\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
127.0.0.1 - - [15/Nov/2025 16:10:41] "POST /api/predict HTTP/1.1" 200 -



Received: cuisine=Chinese, time=Lunch, weather=Cloudy, hunger=Low
Encoded: [np.int64(0), np.int64(3), np.int64(0), np.int64(1)]
Shape: (1, 4)
Prediction: 0, Proba: [0.80445373 0.19554627]


127.0.0.1 - - [15/Nov/2025 16:14:41] "OPTIONS /api/predict HTTP/1.1" 204 -
c:\Users\RACHITH BHARADWAJ T\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
c:\Users\RACHITH BHARADWAJ T\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
127.0.0.1 - - [15/Nov/2025 16:14:41] "POST /api/predict HTTP/1.1" 200 -



Received: cuisine=Italian, time=Lunch, weather=Sunny, hunger=Medium
Encoded: [np.int64(2), np.int64(3), np.int64(2), np.int64(2)]
Shape: (1, 4)
Prediction: 0, Proba: [0.59963157 0.40036843]
